## Gait Video Study
#### Postprocessing the created 3D keypoints (via intrinsic and extrinsic matrix) for lower body and feet and combining together both front and side view 3D coordinates to get the final combined coordinates. Further, scaling all average hip heights to a constant to normalize for subject heights in our dataset.
Reference: http://personal.cityu.edu.hk/meachan/Online%20Anthropometry/Chapter2/Ch2-5.htm

In [21]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML

In [22]:
path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\3D_data\\'

#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

#Dimensions of the image 
w = 800
h = 448

#Bounds for 3D coordinates 
x_min, y_min, z_min = 0.0001, 0.0001, 0.0001
x_max, y_max, z_max = 87, 310, 120

#Hard limits for coordinates, beyond which we delete the respective coordinate before adjusting for 
#z-coordinate or upper bounding for all coordinates 
#We upper bound the z- coordinate between 120 cm(~20 cm higher than the average American hip height) and 180 cm
#(which we assume as our hard upper limit for z-coordinate, beyond which, we assume that coordinate it 
#is recording is not correct, hence we delete that markers' recordings i.e. set x, y, z, confidence = 
#0, 0, 0, 0) to 120 cm. Further later on, to eliminate the size demographics from our datasets, we will 
#scale all coordinates to make sure that the hip height for each subject is same as the 
#average American hip height http://personal.cityu.edu.hk/meachan/Online%20Anthropometry/Chapter2/Ch2-5.htm
limit_x_min, limit_y_min, limit_z_min = -15, -50, -120
limit_x_max, limit_y_max, limit_z_max = 120, 400, 180

#Counts for sanity check after post processing 
x_coord_negative, y_coord_negative, z_coord_negative = 0, 0, 0
global sanity_x, sanity_y, sanity_z

### Utility functions 

In [23]:
def hard_limit(frame_csv):
    #Zero out all x, y, z, and confidence for coordinates that violate the 
    #hard constraints 
    hard_condition = (frame_csv.x>limit_x_max)|(frame_csv.y>limit_y_max)\
    |(frame_csv.z>limit_z_max)|(frame_csv.x<limit_x_min)|(frame_csv.y<limit_y_min)|\
    (frame_csv.z<limit_z_min)
    frame_csv[hard_condition] = 0
    return frame_csv    

In [24]:
def shift_nonnegative(frame_csv):
    #If the minimum z-value is negative, hence a shift is needed 
    #Make sure to not shift the 0 z-coordinate for the markers that are missing 
    #(i.e. have x, y, z, confidence = 0, 0, 0, 0)
    if (frame_csv.x.min()<0): 
        frame_csv.x[frame_csv.confidence!=0]+=(-1*frame_csv.x.min() + x_min)
    if (frame_csv.y.min()<0): 
        frame_csv.y[frame_csv.confidence!=0]+=(-1*frame_csv.y.min() + y_min)
    if (frame_csv.z.min()<0): 
        frame_csv.z[frame_csv.confidence!=0]+=(-1*frame_csv.z.min() + z_min)
    return frame_csv

In [25]:
def upper_bound(frame_csv):
    #Upper bound the x, y, z coordinates if they exceed the max bounds 
    frame_csv.x[frame_csv.x>x_max] = x_max
    frame_csv.y[frame_csv.y>y_max] = y_max
    frame_csv.z[frame_csv.z>z_max] = z_max  
    return frame_csv

In [26]:
def sanity_check(frame_csv):
    #Sanity check to check if there are any x<0, or y<0 or z<0 that 
    #we may need to treat 
    global sanity_x, sanity_y, sanity_z
    if ((frame_csv.x<0).sum()>0 | (frame_csv.x>x_max+x_min).sum()>0):
        snaity_x+=1
        print ('x-coordinate sanity not satisfied for ', frame)
    if ((frame_csv.y<0).sum()>0 | (frame_csv.y>y_max+y_min).sum()>0):
        snaity_y+=1
        print ('y-coordinate sanity not satisfied for ', frame)
    if ((frame_csv.z<0).sum()>0 | (frame_csv.z>z_max+z_min).sum()>0):
        snaity_z+=1
        print ('z-coordinate sanity not satisfied for ', frame)

### Postprocessing for both front (lower body) and side camera (feet)

In [8]:
for cohort in cohorts:
    for trial in trials:
        for camera in cameras:
            frame_path = path+cohort+trial+camera #Path to save the frames to 
            if (os.path.exists(frame_path)):
                videos = os.listdir(frame_path)
#             print (len(videos))
                for video in videos:
                    if not os.path.exists(frame_path+video+'\\processed3d'):
                        os.makedirs(frame_path+video+'\\processed3d')
                    frames = glob.glob(frame_path+video+'\\*.csv')
                    for frame in frames:
    #                     print (frame)
                        csv_path = frame_path+video+'\\processed3d\\'+frame.split('\\')[-1]
                        if not os.path.exists(csv_path):
                            try:
                                frame_csv = pd.read_csv(frame, index_col = 0)
                                #To delete (convert x, y, z, confidence to 0, 0, 0, 0) the coordinates 
                                #which do not satisfy the hard limits and hence 
                                #are most probably not the coordinates for the subject in the frame
                                frame_csv = hard_limit(frame_csv)
                                #To shift up the z-coordinates (and sometime x, y-also)of the frame 
                                #if any z (and sometimes x, y also) is <0 
                                frame_csv = shift_nonnegative(frame_csv)
                                #To upper bound the x>width of the treadmill to the width 
                                # and y>length of the treadmill to the length 
                                frame_csv = upper_bound(frame_csv)
                                #Sanity check to check if there are any x<0, or y<0 or z<0 that 
                                #we may need to treat 
                                if ((frame_csv.x<0).sum()>0):
                                    x_coord_negative+=1
                                    print (frame)
                                if ((frame_csv.y<0).sum()>0):
                                    y_coord_negative+=1
                                    print (frame)
                                if ((frame_csv.z<0).sum()>0):
                                    z_coord_negative+=1
                                    print (frame)
#                                 print (frame_csv)
                                frame_csv.to_csv(csv_path)
                            except Exception as e:
                                print (e)
                    print (video, 'Done!')
print ('Count of negative x, y and z coordinates are:', x_coord_negative, y_coord_negative, z_coord_negative)

InkedGVS_212_T_T1_0_Trim Done!
InkedGVS_212_T_T2_0_Trim Done!
InkedGVS_213_T_T1_0_Trim Done!
InkedGVS_213_T_T2_0_Trim Done!
InkedGVS_214_T_T2_0_Trim Done!
InkedGVS_215_T_T1_0_Trim Done!
InkedGVS_215_T_T2_0_Trim Done!
InkedGVS_216_T_T1_0_Trim Done!
InkedGVS_216_T_T2_0_Trim Done!
InkedGVS_217_T_T1_0_Trim Done!
InkedGVS_217_T_T2_0_Trim Done!
InkedGVS_218_T_T1_0_Trim Done!
InkedGVS_218_T_T2_0_Trim Done!
InkedGVS_219_T_T1_0_Trim Done!
InkedGVS_219_T_T2_0_Trim Done!
InkedGVS_212_T_T1_1_Trim Done!
InkedGVS_212_T_T2_1_Trim Done!
InkedGVS_213_T_T1_1_Trim Done!
InkedGVS_213_T_T2_1_Trim Done!
InkedGVS_214_T_T1_1_Trim Done!
InkedGVS_214_T_T2_1_Trim Done!
InkedGVS_215_T_T1_1_Trim Done!
InkedGVS_215_T_T2_1_Trim Done!
InkedGVS_216_T_T1_1_Trim Done!
InkedGVS_216_T_T2_1_Trim Done!
InkedGVS_217_T_T1_1_Trim Done!
InkedGVS_217_T_T2_1_Trim Done!
InkedGVS_218_T_T1_1_Trim Done!
InkedGVS_218_T_T2_1_Trim Done!
InkedGVS_219_T_T1_1_Trim Done!
InkedGVS_219_T_T2_1_Trim Done!
InkedGVS_212_W_T1_0_Trim Done!
InkedGVS

In [29]:
#Sanity Checks!
#Check that x-coordinate is between 0 and 87 cm (width of the treadmill), y coordinate is between 0 and 310 cm 
#(length of the treadmill) and z-coordinate is between 0 and ~120 (maximum length of subject's lower body)
sanity_x, sanity_y, sanity_z = 0, 0, 0
for cohort in cohorts:
    for trial in trials:
        for camera in cameras:
            frame_path = path+cohort+trial+camera #Path to save the frames to 
            if (os.path.exists(frame_path)):
                videos = os.listdir(frame_path)
#             print (len(videos))
            for video in videos:
                frames = glob.glob(frame_path+video+'\\processed3d\\*.csv')
                for frame in frames:
#                     print (frame)
                    frame_csv = pd.read_csv(frame, index_col = 0)
                    sanity_check(frame_csv)
                print (video, 'Done!')
                print ('Sanities (x, y, z in that order) now:', sanity_x, sanity_y, sanity_z)
print ('Count of violating sanity x, y and z coordinates are: ', sanity_x, sanity_y, sanity_z)

InkedGVS_212_T_T1_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_212_T_T2_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_213_T_T1_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_213_T_T2_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_214_T_T2_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_215_T_T1_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_215_T_T2_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_216_T_T1_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_216_T_T2_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_217_T_T1_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_217_T_T2_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_218_T_T1_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_218_T_T2_0_Trim Done!
Sanities (x, y, z in that order) now: 0 0 0
InkedGVS_219_T_T1_0_Trim 

### Combine together both front and side view to get final 3D coordinates 

In [ ]:
#Make sure to give feet frames much lesser weight that lower body frames 


In [ ]:
#After combining, scale all average hip heights to a constant to normalize for subject heights in our dataset
#This is to make sure that we eliminate the effects of size demographics from our dataset 
#Refer http://personal.cityu.edu.hk/meachan/Online%20Anthropometry/Chapter2/Ch2-5.htm


In [1]:
#Check if there are any missing values (x, y, z, confidence = 0, 0, 0, 0) in this combined data 
#and how can we treat them before feeding in the models 


In [ ]:
#Patterns and Sanity checks!
#In combined data, after posprocessing for lower body done, take a look at trend of z-coordinate especially to 
#check the hip, knee and ankle height make sense w.r.t norms and similarly some other sanity checks 
#like distance between toe and heel etc.
